In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

from ipywidgets.widgets import interact, IntSlider, FloatSlider
from IPython.display import display

from KSS_curve import * 
from kss import *

# Pfade zu den HDF5-Daten

In [ ]:
base_path = '/Users/spede/Desktop/KSS/'
cm_path = os.path.join(base_path, 'raw_cm.hdf5')
dmvt_path = os.path.join(base_path, 'raw_dmvt.hdf5')

# Kurvenverläufe für eine Leitgüte von 12400

In [ ]:
cm = CurveManager.load(cm_path)
key = 'DST_Temp_vor_STB'
indexNoZeros = cm.index[cm.index['DST_Temp_vor_STB'] != 0]
selection = indexNoZeros.query('DWLD_LEITGUETE==12400')
curves = cm.load_curves(key, selection)
#all_data = [curve for curve in curves]

# Zufällig gewählte Kurvenverläufe

In [ ]:
cm.plot_random(amount=10, dmvt_id='DST_Temp_vor_STB')
_=plt.title('Zufällig gewählte Kurvenverläufe')

# Bestimmen von Start und Ende

In [ ]:
@interact(data_idx=IntSlider(min=1, max=len(all_data), step=1, value=1))
def plot_start_end(data_idx):
    
    # Filterlänge des Tiefpasses (Mittelwertbilder/Integration) und des Hochpasses (Differenzierung)
    filt_len = 3
    
    # Fensterlänge zur Bestimmung der Streuung
    window_size = 10
    
    # Kurvenverlauf zu vorliegendem Index
    data = all_data[data_idx][1]

    # Glätten des Kurvenverlaufs
    data_smoothed = smooth_data(data, filt_len)
    
    # Berechnen der Streuung pro Fensterlänge
    data_std = windowed_std(data, window_size)
    
    # Differenzierung der Streuung und Unterteilung der Bereiche (Startbereich/Endbereich)
    diff_start, diff_end = get_diff(data_std, window_size, filt_len)
    
    # Start- und Endstelle bestimmen
    start = int((np.argmin(diff_start) + filt_len - 1) * window_size) + filt_len - 1
    end = int((np.argmax(diff_end) + 150/window_size) * window_size) + filt_len - 1

    # Plots
    plt.figure(figsize=(20,3))
    ax_1 = plt.subplot(121)
    ax_1.plot(data)
    ax_1.plot(start, data[start], 'ro')
    ax_1.plot(end, data[end], 'go')
    ax_1.set_title('Kurvenverlauf mit Start- und Endpunkten')
    ax_1.set_xlabel('Segment/n')
    ax_1.set_ylabel('DST_Temp_vor_STB')

    ax_2 = plt.subplot(122)
    ax_2.plot(data_std)
    ax_2.plot([150/window_size, 150/window_size], [0, max(data_std)], 'r')
    ax_2.set_title('Gefensterte Streuung')
    ax_2.set_xlabel('Segment/(n*Fensterlänge)')
    ax_2.set_ylabel('Streuung')
    
    plt.figure(figsize=(20,3))
    ax_1 = plt.subplot(121)
    ax_1.plot(diff_start)
    ax_1.set_title('1. Ableitung der Streuung des Startbereichs')
    ax_1.set_xlabel('Segment/(n*Fensterlänge)')
    ax_1.set_ylabel('1. Ableitung')
    

    ax_2 = plt.subplot(122)
    ax_2.plot(diff_end)
    ax_2.set_title('1. Ableitung der Streuung des Endbereichs')
    ax_2.set_xlabel('Segment/(n*Fensterlänge)')
    ax_2.set_ylabel('1. Ableitung')

# Histogramm der Messwerte im Start- bis Endbereich

In [ ]:
all_cropped_data = []

for _, data in all_data:
    start, end = get_start_end(data, 10, 3)
    all_cropped_data += list(data[start:end])

In [ ]:
plt.figure(figsize=(15,5))
plt.hist(all_cropped_data, bins=100, normed=True)
plt.title('Histogramm der Messwerte im Start- bis Endbereich')
plt.xlabel('DST_Temp_vor_STB')
_=plt.ylabel('relatie Häufigkeit')